In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import random

In [2]:
from platform import python_version

print(python_version())

3.7.6


In [3]:
#Data from https://snap.stanford.edu/data/ego-Facebook.html
H = nx.read_edgelist("facebook_combined.txt", nodetype=int)

In [4]:
list_of_events = []

In [5]:
times, S, I, R = [0], [len(H.nodes)], [0], [0]

In [6]:
Tmax = 10

In [7]:
#Transmission rate: beta
#Recovery rate: gamma

beta, gamma = 1.5, 1

In [8]:
for u in H.nodes:
    H.nodes[u]['status'] = 'susceptible'
    H.nodes[u]['pred_inf_time'] = np.inf

In [9]:
#initial infected
u = np.random.choice(H.nodes, size=1)[0]

H.nodes[u]['pred_inf_time'] = 0

#events are (time, node, type)
event = (0, u, 'transmit')

list_of_events.append(event)
list_of_events.sort()

In [10]:
def process_recovery(event):
    global H, times, S, I, R
    #events are (0: time, 1: node, 2: type)

    H.nodes[event[1]]['status'] = 'recovered'
    times.append(event[0])
    S.append(S[-1])
    I.append(I[-1]-1)
    R.append(R[-1]+1)

In [11]:
def process_transmission(event):
    global H, times, S, I, R, list_of_events, beta, gamma, Tmax
    
    #events are (0: time, 1: node, 2: type)
    if H.nodes[event[1]]['status'] == 'susceptible':

        H.nodes[event[1]]['status'] = 'infected'

        times.append(event[0])
        S.append(S[-1]-1)
        I.append(I[-1]+1)
        R.append(R[-1])

        rec_time = event[0] + random.expovariate(gamma)

        if rec_time < Tmax:
            new_event = (rec_time, event[1], 'recovery')
            list_of_events.append(new_event)
            list_of_events.sort()

        for target in H.neighbors(event[1]):
            if H.nodes[target]['status'] == 'susceptible':
                inf_time = event[0] + random.expovariate(beta)

                if inf_time < min(rec_time, H.nodes[target]['pred_inf_time'], Tmax):
                    new_event = (inf_time, target, 'transmit')
                    list_of_events.append(new_event)
                    list_of_events.sort()
                    H.nodes[target]['pred_inf_time'] = inf_time            

In [12]:
while(len(list_of_events) > 0):
    event = list_of_events.pop(0)
    
    if event[2] == 'transmit':
        process_transmission(event)
    else:
        process_recovery(event)

In [13]:
network_data = np.column_stack((times, S, I, R))

In [14]:
pd.DataFrame(network_data).to_csv("network_data.csv")